# Load and create data sets

Load data.

In [2]:
### LINES TO IMPORT THE DECONFOUNDER PACKAGE IN THE PARENT FOLDER ###
import os
import sys
sys.path.append("..")
### IMPORTS
from deconfounder.causal_tree import CausalTree
from deconfounder.causal_forest import CausalForest
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import numpy as np
from numpy.polynomial.polynomial import polyfit
import pandas as pd
import scipy.stats as st
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
import time

# Load data
df = pd.read_csv("../data/criteo-uplift-v2.1.csv")
df = df.sample(frac=1, random_state=42)
all_features = df.columns.values[:12].tolist()

Find the feature that is correalted the most with the outcome and remove that feature (i.e., the confounder).

In [3]:
corr_matrix = df[all_features + ['visit']].corr()
confounder = corr_matrix['visit'][:-1].sort_values().index.values[-1]
print(confounder)
limited_features = list(all_features)
limited_features.remove(confounder)

f9


Obtain observational data. Drop observations using the confounder.

In [4]:
def print_outcomes(data):
    means = data.groupby('treatment').visit.mean()
    print(f"Size of data set: {data.shape[0]:,}")
    print(f"Avg. Outcome (treated): {np.round(means.loc[1]*100, 2)}%")
    print(f"Avg. Outcome (control): {np.round(means.loc[0]*100, 2)}%")
    print(f"Estimated avg. effect: {np.round((means.loc[1]-means.loc[0])*100, 2)}%")

train_frac = 0.5
train_size = int(df.shape[0]*train_frac)
obs_df = df[:train_size]
train_df = df[:train_size]
drop_frac = 0.1

conf_ranking = obs_df[confounder].rank(method="first")
keep =  ((obs_df.treatment == 1) & (conf_ranking > int(train_size * drop_frac))) | \
        ((obs_df.treatment == 0) & (conf_ranking < int(train_size * (1 - drop_frac))))
obs_df = train_df[keep]
print("Observational (naive) setup")
print_outcomes(obs_df)
print("-------------")
exp_df = train_df.sample(n=obs_df.shape[0],random_state=42)
print("Experimental setup")
print_outcomes(exp_df)
print("-------------")
print("Entire data")
print_outcomes(df)

Observational (naive) setup
Size of data set: 6,297,527
Avg. Outcome (treated): 5.31%
Avg. Outcome (control): 1.23%
Estimated avg. effect: 4.08%
-------------
Experimental setup
Size of data set: 6,297,527
Avg. Outcome (treated): 4.86%
Avg. Outcome (control): 3.8%
Estimated avg. effect: 1.06%
-------------
Entire data
Size of data set: 13,979,592
Avg. Outcome (treated): 4.85%
Avg. Outcome (control): 3.82%
Estimated avg. effect: 1.03%


Build causal tree using observational data. Control for observable confounding.

In [5]:
def fit_tree(df_causal, tree_type=CausalForest, min_samples_leaf=5000, tune=False, params=None, scoring=None, verbose=True):    
    X = df_causal[limited_features + ['treatment']].rename(columns={"treatment":"treated"})
    y =  df_causal.visit
    # Fit and tune causal tree
    start_time = time.time()
    if tune:
        if params is None:
            params = range(1000, 10000, 1000)
        tuned_parameters = [{'min_samples_leaf': params}]
        grid_tree = GridSearchCV(tree_type(random_state=42, n_estimators=10), tuned_parameters, cv=4, 
                                 verbose=10, n_jobs=4, scoring=scoring)
        grid_tree.fit(X, y)
        print("Best parameters set found on development set:")
        print(grid_tree.best_params_)
        tree_model = grid_tree.best_estimator_
    else:
        tree_model = tree_type(min_samples_leaf=min_samples_leaf, random_state=42, n_estimators=10)
        tree_model.fit(X, y)
    if verbose:
        print("--- Time to fit (and tune) causal tree %s seconds ---" % (time.time() - start_time))
    return tree_model

#obs_tree = fit_tree(obs_df, tune=True, params=[8000, 16000, 32000, 64000, 128000])
#Best parameters set found on development set:
#{'min_samples_leaf': 32000}
#--- Time to fit (and tune) causal tree 1082.7572462558746 seconds ---
#obs_tree = fit_tree(obs_df, min_samples_leaf=50000)

In [6]:
limited_exp = exp_df.sample(50000,random_state=42)
limited_tree = fit_tree(limited_exp, tune=True, params=np.arange(1, 11)*1000)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
Best parameters set found on development set:
{'min_samples_leaf': 1000}
--- Time to fit (and tune) causal tree 5.552044630050659 seconds ---


In [7]:
curve_sizes = np.array([50000, 100000, 200000, 400000])
for exp_size in curve_sizes:
    subset_exp = exp_df.sample(exp_size,random_state=42)
    subset_tree = fit_tree(subset_exp, tune=False, min_samples_leaf=limited_tree.min_samples_leaf)

--- Time to fit (and tune) causal tree 0.6478450298309326 seconds ---
--- Time to fit (and tune) causal tree 1.7934596538543701 seconds ---
--- Time to fit (and tune) causal tree 5.774129629135132 seconds ---
--- Time to fit (and tune) causal tree 14.570174932479858 seconds ---


In [ ]:
limited_tree.min_samples_leaf

In [ ]:
fit_tree(limited_exp, tune=False, min_samples_leaf=limited_tree.min_samples_leaf)